# Import Libraries

In [2]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, make_scorer, mean_squared_error
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import randint, uniform

In [3]:
df1 = pd.read_csv("/Users/verapinto/Desktop/Final_Project/Data_Science_E2E_Project/Forecast.csv")
df1.head()

,Month-Year,PRODUCT_FAMILY,GMID,LOC,Country,Sales,Forecast,MAPE,MAPE_Impact,MAPE_Contribution,Absolute Error,Non Absolute Error
0,23-Jul,6,600001,KR001,KR,909,868.279179,4.50%,0.00%,0.00%,40.720821,-40.720821
1,23-Aug,6,600001,KR001,KR,786,867.675973,10.40%,0.01%,0.00%,81.675973,81.675973
2,23-Sep,6,600001,KR001,KR,846,906.330589,7.10%,0.01%,0.00%,60.330590,60.330590
3,23-Oct,6,600001,KR001,KR,761,907.327425,19.20%,0.01%,0.10%,146.327425,146.327425
4,23-Nov,6,600001,KR001,KR,786,868.000000,10.40%,0.01%,0.00%,82.000000,82.000000


In [6]:
df1.shape

(18159, 12)

In [8]:
df2 = pd.read_csv("/Users/verapinto/Desktop/Final_Project/Data_Science_E2E_Project/Stocks.csv")
df2.head()

,Product.Family,GMID,GMID_Description,Market,Geography,Material.Type.GMID,Date,Coverage,Min,Max
0,6,600001,06 2.9MG/5ML,Korea,International,FG,23-Jul,1.82,2.73,4.91
1,6,600002,06 2.9MG/5ML,Hong Kong,Greater China,FG,23-Jul,3.09,2.86,5.68
2,6,600003,06 2.9MG/5ML,Canada,North America,FG,23-Jul,3.41,2.73,5.68
3,6,600004,06 2.9MG/5ML,Malaysia,International,FG,23-Jul,0.91,3.18,9.14
4,6,600005,06 500IU/5ML,Australia,International,FG,23-Jul,0.91,2.73,4.59


In [ ]:
df2.shape

In [ ]:
df_geo  = pd.read_csv("/Users/verapinto/Desktop/Final_Project/Data_Science_E2E_Project/Countries_Geo.csv")
df_geo.head()

In [ ]:
df_md  = pd.read_csv("/Users/verapinto/Desktop/Final_Project/Data_Science_E2E_Project/MD.csv")
df_md.head()

# Data Preparation

### Merge Datasets

#### Merge df1 ( sales, forecast, mape, absolute error information) with df_geo to append Market and Geography column

In [ ]:
# Merge df1 with country metadata to add Country_Desc and Geography columns
df1_enriched = df1.merge(
    df_geo[['Country_Key', 'Country_Desc', 'Geography']],
    left_on='Country',
    right_on='Country_Key',
    how='left'  # Preserves all rows from df1
)
df1_enriched.drop(columns='Country_Key', inplace=True)
df1_enriched.head()

In [ ]:
# check duplication of rows after merge
print("Before merge:", len(df1))
print("After merge:", len(df1_enriched))

In [ ]:
# Change column Month-Year to Date in df1 and df2
df1_enriched.rename(columns={'Month-Year': 'Date'}, inplace=True)

# Standardize keys in both dataframes
df1_enriched['Date'] = df1_enriched['Date'].astype(str).str.strip().str.strip("'").str.title()
df1_enriched['Date'] = pd.to_datetime(df1_enriched['Date'], format='%y-%b', errors='coerce')

df2['Date'] = df2['Date'].astype(str).str.strip().str.title()
df2['Date'] = pd.to_datetime(df2['Date'], format='%y-%b', errors='coerce')

#### Identify and resolve duplicate records in df2

In [ ]:
#check duplicates
# Check if GMID + Date + Country_Key appear only once
duplicates = df2.duplicated(subset=['GMID', 'Date', 'Market'])
print("Duplicate rows in df2:", duplicates.sum())


In [ ]:
#Define key columns and priority rows (non-null coverage, min, max)

# Replace NaNs with 0 for comparison
df2['Coverage'] = df2['Coverage'].fillna(0)
df2['Min'] = df2['Min'].fillna(0)
df2['Max'] = df2['Max'].fillna(0)

# Mark rows with valid (non-zero) values
df2['has_data'] = (df2['Coverage'] > 0) | (df2['Min'] > 0) | (df2['Max'] > 0)

#Sort to valid rows come first
df2 = df2.sort_values(by='has_data', ascending=False)

#Drop duplicates by keeping the first one (which now has valid values)
df2 = df2.drop_duplicates(subset=['GMID', 'Date', 'Market'], keep='first')

#Drop helper column
df2.drop(columns='has_data', inplace=True)

In [ ]:
# double check duplicated rows
df2.duplicated(subset=['GMID', 'Date', 'Market']).sum()

#### Merge df1 (main dataset) with df2 (stock levels information)

In [ ]:
# Merge 2 tables
df_merged = df1_enriched.merge(
    df2[['GMID', 'Date', 'Market', 'Coverage', 'Min', 'Max']],
    left_on=['GMID', 'Date', 'Country_Desc'],
    right_on=['GMID', 'Date', 'Market'],
    how='left'
)

df_merged.drop(columns=['MAPE_Impact','MAPE_Contribution', 'Non Absolute Error', 'Market'], inplace=True)
df_merged.head()

In [ ]:
#check duplication of rows after merge
print("Before merge:", len(df1_enriched))
print("After merge:", len(df_merged))


#### Merge df1 (main dataset) with df3 (market and product segmentation information)

In [ ]:
df_smart = df_md[['PRODUCT_FAMILY', 'SMART_SEGMENTATION']].drop_duplicates()

df1_smart = df_merged.merge(
    df_smart,
    on='PRODUCT_FAMILY',
    how='left'
)
df1_smart.head()

In [ ]:
#check duplication of rows after merge
print("Before merge:", len(df_merged))
print("After merge:", len(df1_smart))

In [ ]:
df_mco = df_md[['Market', 'MCO_SEGMENTATION']].drop_duplicates()

df1_mco = df1_smart.merge(
    df_mco,
    left_on='Country',
    right_on='Market',
    how='left'
).drop(columns=['Market'])  # Optional: remove Market after merge
df1_mco.head()

In [ ]:
#check duplication of rows after merge
print("Before merge:", len(df_merged))
print("After merge:", len(df1_mco))

In [ ]:
df_strategy = df_md[['GMID', 'Market', 'STRATEGY_SEGMENTATION']].drop_duplicates()

df_final = df1_mco.merge(
    df_strategy,
    left_on=['GMID', 'Country'],
    right_on=['GMID', 'Market'],
    how='left'
).drop(columns=['Market'])  # Again, drop Market if it’s redundant
df_final.head()

In [ ]:
#check duplication of rows after merge
print("Before merge:", len(df_merged))
print("After merge:", len(df_final))


In [ ]:
# rename columns name

df_final.rename(columns={'Country_Desc': 'Market'}, inplace=True)
df_final.rename(columns={'SMART_SEGMENTATION': 'pf_segmentation'}, inplace=True)
df_final.rename(columns={'MCO_SEGMENTATION': 'market_segmentation'}, inplace=True)
df_final.rename(columns={'STRATEGY_SEGMENTATION': 'gmid_segmentation'}, inplace=True)
df_final.rename(columns={'Coverage': 'coverage_months'}, inplace=True)
                         
# Define the new order of columns
new_order = ['Date', 'PRODUCT_FAMILY', 'GMID', 'LOC', 'Country','Market', 'Geography', 'Sales','Forecast', 'MAPE',
             'Absolute Error','coverage_months', 'Min', 'Max', 'market_segmentation', 'pf_segmentation',	'gmid_segmentation']

# Apply the new order
df = df_final[new_order]
df.head()

# Data Cleaning & Wrangling

In [ ]:
#format the headers
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')


In [ ]:
#check data types

df.info()

In [ ]:
#change types by column

# Remove commas and convert to integers
df['sales'] = df['sales'].astype(str).str.replace(',', '', regex=False)
df['sales'] = pd.to_numeric(df['sales'], errors='coerce').fillna(0).astype('int64')

df['forecast'] = (
    df['forecast']
    .astype(str)
    .str.replace(',', '', regex=False)
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
    .apply(lambda x: int(round(x)))
)


float_cols = ['mape','absolute_error','coverage_months', 'min', 'max']

for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').round(2)

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [ ]:
#recalculate MAPE, Absolute error columns

# Absolute errors
df['absolute_error'] = (df['forecast'].astype(float) - df['sales'].astype(float)).abs().round(2)

# Recalculate mape 
df['mape'] = df.apply(
    lambda row: round(abs(row['sales'] - row['forecast']) / row['sales'] * 100, 2)
    if row['sales'] != 0 else 0,
    axis=1
)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
#Count of missing values per column
df.isna().sum()

In [ ]:
# replace nan values
df[['coverage_months', 'min', 'max']] = df[['coverage_months', 'min', 'max']].fillna(0).round(2)


In [ ]:
df.isna().sum()

In [ ]:
#check gmid_key duplicates
df['gmid_key'] = df['gmid'].astype(str) + '_'+ df['year'].astype(str) + df['month'].astype(str) + '_'+ df['country'].astype(str)
duplicate_rows = df['gmid_key'].duplicated().sum()
duplicate_rows


In [ ]:
df[df['gmid_key'].duplicated()]


In [ ]:
#Handle duplicated gmid_key rows by summing sales and forecast, while keeping the first values for the rest, 
#The difference is in loc and the country is the same (FR001 and FR003; Ae001 and AE002, SG001 and SG002)

agg_dict = {
    'sales': 'sum',
    'forecast': 'sum',
    'date': 'first',
    'gmid': 'first',
    'country': 'first',
    'product_family': 'first',
    'market': 'first',
    'geography': 'first',
    'mape': 'first',
    'absolute_error': 'first',
    'coverage_months': 'first',
    'min': 'first',
    'max': 'first',
    'market_segmentation': 'first', 
    'pf_segmentation': 'first',
    'gmid_segmentation': 'first',
    'year': 'first',
    'month': 'first'
}
df_grouped = df.groupby('gmid_key', as_index=False).agg(agg_dict)


In [ ]:
#check duplicates
df_grouped['gmid_key'].duplicated().sum()

In [ ]:
# Define the new order of columns
new_order = ['gmid_key','date','year', 'month','product_family', 'gmid', 'country','market', 'geography', 'sales','forecast', 'mape',
             'absolute_error','coverage_months', 'min', 'max', 'market_segmentation', 'pf_segmentation', 'gmid_segmentation']

# Apply the new order
df_all = df_grouped[new_order]


In [ ]:
#recalculate MAPE and Absolute error columns

# Absolute errors
df_all['absolute_error'] = (df_all['forecast'].astype(float) - df_all['sales'].astype(float)).abs().round(2)

# Recalculate mape 
df_all['mape'] = df_all.apply(
    lambda row: round(abs(row['sales'] - row['forecast']) / row['sales'] * 100, 2)
    if row['sales'] != 0 else 0,
    axis=1
)

In [ ]:
df_all.head()

In [ ]:
df_all.to_csv('file.csv', index=False)

# Exploratory Data Analysis (EDA)

In [ ]:
#count of GMIDs
num_gmids = df_all['gmid'].nunique()
print(f"Total unique GMIDs: {num_gmids}")


In [ ]:
#count of Markets
num_markt = df_all['country'].nunique()
print(f"Total unique Market: {num_markt}")

In [ ]:
#Check statistical summary
num_cols = ['sales', 'forecast', 'mape','absolute_error']

df_all[num_cols].describe()


### Univariate Analysis

In [ ]:
#Check distribution for numerical features
metrics = ['sales', 'forecast', 'mape']

for metric in metrics:
    plt.figure(figsize=(12, 5))
    
    # Histogram 
    plt.subplot(1, 2, 1)
    sns.histplot(df_all[metric], bins=40, color='skyblue')
    plt.title(f'{metric} - Histogram')

    # Boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df_all[metric], color='salmon')
    plt.title(f'{metric} - Boxplot')

    plt.tight_layout()
    plt.show()



In [ ]:
#High Sales by Country
# Filter rows where sales > 75th percentile
filtered_df = df_all[df_all['sales'] > 650]

# Group by country
country_counts = filtered_df['country'].value_counts().reset_index()
country_counts.columns = ['country', 'count']

# Plot
plt.figure(figsize=(15, 5))
sns.barplot(data=country_counts, x='country', y='count', palette='viridis')
plt.title('Number of Records with Sales > 650 by Country')
plt.xlabel('Country')
plt.ylabel('Record Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
#High Sales by Product Family

# Group by product_family
product_counts = filtered_df['product_family'].value_counts().reset_index()
product_counts.columns = ['product_family', 'count']

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=product_counts, x='product_family', y='count', palette='magma')
plt.title('Number of Records with Sales > 650 by Product Family')
plt.xlabel('Product Family')
plt.ylabel('Record Count')
plt.tight_layout()
plt.show()


In [ ]:
# Filter rows where MAPE > 75th percentile
high_mape_df = df_all[df_all['mape'] > 51.145]

# Count high MAPE records by country
country_mape_counts = high_mape_df['country'].value_counts().reset_index()
country_mape_counts.columns = ['country', 'count']

# Plot
plt.figure(figsize=(16, 5))
sns.barplot(data=country_mape_counts, x='country', y='count', palette='rocket')
plt.title('Records with MAPE > 75th Percentile by Country')
plt.xlabel('Country')
plt.ylabel('Record Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# Filter rows where MAPE > 1000%
high_mape_df2 = df_all[df_all['mape'] > 1000]

# Count high MAPE records by country
country_mape_counts2 = high_mape_df2['country'].value_counts().reset_index()
country_mape_counts2.columns = ['country', 'count']

# Plot
plt.figure(figsize=(16, 5))
sns.barplot(data=country_mape_counts2, x='country', y='count', palette='rocket')
plt.title('Records with MAPE > 1000 by Country')
plt.xlabel('Country')
plt.ylabel('Record Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Count the number of entries per country
country_counts = df_all['country'].value_counts().sort_values(ascending=True)

# Set the figure size for better spacing
plt.figure(figsize=(16, 8))  # Dynamically scale height

# Plot horizontal bar chart
sns.barplot(x=country_counts.index, y=country_counts.values, palette="viridis")

# Beautify the chart
plt.title("Number of Records per Country", fontsize=16)
plt.xlabel("Count", fontsize=12)
plt.ylabel("Country", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

### Bivariate Analysis

In [ ]:
#mean (average) of sales for each month
sns.barplot(x="month", y="sales", data=df_all, ci=None)

In [ ]:
df_all.set_index('date').resample('M')['sales'].sum().plot(title='Monthly Sales Trend')


* Despite the upward trend, there are visible dips along the way, suggesting seasonal effects, market shifts, or temporary challenges.

In [ ]:
#Sales Vs Forecast
sns.scatterplot(x='forecast', y='sales', data=df_all)
plt.title('Forecast vs. Sales')


* Strong Positive Correlation: Sales values rise alongside forecast values. This tight clustering along a diagonal suggests a reliable prediction model—when forecasts go up, actual sales usually follow.

* Forecast Accuracy: The proximity of most data points to the diagonal implies high accuracy in predictions. Few outliers indicate minimal deviation between forecasted and actual results.

* Strategic Alignment: The consistent relationship means business strategies based on forecasts are likely well-aligned with market performance.

In [ ]:
#sales vs. forecast side by side for each product family
# Group and sum sales & forecast by product family
sales_forecast_pf = df_all.groupby('product_family')[['sales', 'forecast']].sum().reset_index()
# Melt data to long format for grouped bars
melted_pf = sales_forecast_pf.melt(
    id_vars='product_family',
    value_vars=['sales', 'forecast'],
    var_name='type',
    value_name='value'
)



plt.figure(figsize=(12, 6))
ax = sns.barplot(data=melted_pf, x='product_family', y='value', hue='type', palette='Set2')

# Use logarithmic scale for y-axis
ax.set_yscale('log')




plt.title('Sales vs. Forecast by Product Family (Log Scale)')
plt.xlabel('Product Family')
plt.ylabel('Total Units (log scale)')
plt.legend(title='Metric')
plt.tight_layout()
plt.show()



In [ ]:
# Group and calculate average MAPE by product family
mape_by_pf = df_all.groupby('product_family')['mape'].mean().reset_index()

# Sort by MAPE if needed (optional)
mape_by_pf = mape_by_pf.sort_values(by='mape', ascending=False)

# Plot
plt.figure(figsize=(12, 5))
ax = sns.barplot(data=mape_by_pf, x='product_family', y='mape', palette='Blues_r')

# Add value labels
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge')

plt.title('Average MAPE by Product Family')
plt.xlabel('Product Family')
plt.ylabel('Average MAPE (%)')
plt.tight_layout()
plt.show()



In [ ]:
# Sum total sales per country
sales_by_country = df_all.groupby("country", as_index=False)["sales"].sum()
sales_by_country = sales_by_country.sort_values(by="sales", ascending=False)

# Plot total sales
plt.figure(figsize=(16, 8))
sns.barplot(x="country", y="sales", data=sales_by_country)
plt.title("Total Sales by Country in volume", fontsize=16)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
#mape across Countries

# First, extract the sorted country order based on total sales
country_order = sales_by_country['country']

# Now use that order in your MAPE boxplot
plt.figure(figsize=(16, 6))
sns.boxplot(x='country', y='mape', data=df_all, order=country_order)
plt.title("MAPE by Country", fontsize=16)
plt.xlabel("Country", fontsize=12)
plt.ylabel("MAPE", fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# Group by year and month to calculate average MAPE
mape_monthly_yearly = (
    df_all.groupby(['year', 'month'])['mape']
    .mean()
    .reset_index()
)

# Optional: create a datetime column for plotting
mape_monthly_yearly['date'] = pd.to_datetime(
    mape_monthly_yearly[['year', 'month']].assign(day=1)
)


plt.figure(figsize=(14, 6))
sns.lineplot(data=mape_monthly_yearly, x='date', y='mape', marker='o', color='darkgreen')

plt.title('Average MAPE Evolution Across Months and Years')
plt.xlabel('Date')
plt.ylabel('Average MAPE (%)')
plt.grid(True)
plt.tight_layout()
plt.show()


#### Diagnose the April 2024 Spike

In [ ]:
spike_df = df_all[(df_all['year'] == 2024) & (df_all['month'] == 4)]
spike_summary = spike_df.groupby(['country', 'product_family'])['mape'].mean().reset_index().sort_values(by='mape', ascending=False)

spike_summary

In [ ]:
geo_mape = df_all.groupby(['geography', 'year', 'month'])['mape'].mean().reset_index()

geo_mape

In [ ]:
# Create a datetime column for proper x-axis formatting
geo_mape['date'] = pd.to_datetime(geo_mape[['year', 'month']].assign(day=1))

plt.figure(figsize=(14, 6))
sns.lineplot(data=geo_mape, x='date', y='mape', hue='geography', marker='o', palette='Set1')

plt.title('Average MAPE over Time by Geography')
plt.xlabel('Date')
plt.ylabel('Average MAPE (%)')
plt.legend(title='Geography', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_all[['mape', 'coverage_months', 'sales', 'absolute_error']].corr()


In [ ]:
# Optional visualization: scatterplot
sns.scatterplot(data=df_all, x='sales', y='absolute_error', hue='geography')


In [ ]:
# Group by year and month to calculate average MAPE
monthly_mape = (
    df_all.groupby(['year', 'month'])['mape']
    .mean()
    .reset_index()
    .sort_values(by='mape', ascending=False)
)

# Display top months
top_mape_months = monthly_mape.head(10)
print(top_mape_months)


In [ ]:
# Top 10 months by average MAPE
plt.figure(figsize=(10, 5))
monthly_top = top_mape_months.copy()
monthly_top['label'] = monthly_top['year'].astype(str) + '-' + monthly_top['month'].astype(str).str.zfill(2)

sns.barplot(data=monthly_top, x='label', y='mape', palette='Reds_r')
plt.title('Months with Highest Average MAPE')
plt.xlabel('Month')
plt.ylabel('MAPE (%)')
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.2f')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# Group by country and calculate average MAPE
country_mape = (
    df_all.groupby('country')['mape']
    .mean()
    .reset_index()
    .sort_values(by='mape', ascending=False)
)

# Display top 5 countries
top_5_countries = country_mape.head(5)
print(top_5_countries)


In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(data=top_5_countries, x='country', y='mape', palette='Oranges_r')
plt.title('Top 5 Countries with Highest Average MAPE')
plt.xlabel('Country')
plt.ylabel('MAPE (%)')
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.2f')
plt.tight_layout()
plt.show()


In [ ]:
# Group by product family and calculate MAPE standard deviation
pf_erratic = (
    df_all.groupby('product_family')['mape']
    .std()
    .reset_index()
    .rename(columns={'mape': 'mape_std'})
    .sort_values(by='mape_std', ascending=False)
)

# Display top erratic product families
top_erratic_pf = pf_erratic.head(10)
print(top_erratic_pf)


In [ ]:
plt.figure(figsize=(12, 5))
sns.barplot(data=top_erratic_pf, x='product_family', y='mape_std', palette='Purples')
plt.title('Product Families with Most Erratic Forecasts (MAPE Std Dev)')
plt.xlabel('Product Family')
plt.ylabel('MAPE Std Dev')
for container in plt.gca().containers:
    plt.gca().bar_label(container, fmt='%.2f')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate average MAPE by country and sort descending
data = (
    df_all.groupby('country')['mape']
    .mean()
    .reset_index()
    .sort_values(by='mape', ascending=False)
)

# Plot bar chart
plt.figure(figsize=(16, 6))
ax = sns.barplot(data=data, x='country', y='mape', palette='viridis')

# Add value labels to bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', label_type='edge')

plt.title('Average MAPE by Country (Descending Order)')
plt.xlabel('Country')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
#mape across Product Families

plt.figure(figsize=(14, 6))  # You can adjust width and height as needed
sns.boxplot(x='product_family', y='mape', data=df_all)
plt.title("MAPE Across Product Families", fontsize=16)
plt.xlabel("Product Family", fontsize=12)
plt.ylabel("MAPE", fontsize=12)
plt.xticks(rotation=0)  # Rotating labels often helps with wider plots
plt.tight_layout()
plt.show()



In [ ]:
#mape by Geography

plt.figure(figsize=(14, 6))  # You can adjust width and height as needed
sns.boxplot(x='geography', y='mape', data=df_all)
plt.title("MAPE By Geography", fontsize=16)
plt.xlabel("Geography", fontsize=12)
plt.ylabel("MAPE", fontsize=12)
plt.xticks(rotation=0)  # Rotating labels often helps with wider plots
plt.tight_layout()
plt.show()

In [ ]:
#Correlation Heatmap
correlation = df_all[['year','month','product_family','gmid','sales', 'forecast', 'mape', 'absolute_error']].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')


In [ ]:
def count_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return ((series < lower) | (series > upper)).sum()

# Columns to analyze
cols_to_check = ['sales', 'forecast', 'mape']
outlier_counts = {col: count_outliers_iqr(df_all[col].dropna()) for col in cols_to_check}

# Display
pd.Series(outlier_counts, name='Outlier Count')


* sales - around 12% - Moderate; could cap or transform.  
* forecast - around 12% - Similar to sales; likely due to erratic predictions.
* mape - around 6% - Many MAPE outliers reflect high error (often from small sales).

### Log transformation and segmentation

In [ ]:
#consider only sales >0

df_active = df_all[df_all['sales'] > 0].copy()


# Log transformation and segmentation
df_active['log_sales'] = np.log1p(df_active['sales']) 
df_active['log_segment'] = pd.cut(df_active['log_sales'], 
                                 bins=[-np.inf, 2, 4, 6, np.inf], 
                                 labels=['Very Low', 'Low-Mid', 'Mid-High', 'Very-High'])

# Plotting both segmentation strategies side-by-side
plt.figure(figsize=(14, 6))

# Log-based plot
plt.subplot(1, 2, 2)
sns.countplot(x='log_segment', data=df_active, palette='summer')
plt.title('Sales Segmentation by Log Scale')
plt.xlabel('Segment')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Plot original vs log-transformed sales
plt.figure(figsize=(14, 6))

# Original sales distribution
plt.subplot(1, 2, 1)
sns.histplot(df_all['sales'], bins=50, kde=True, color='skyblue')
plt.title('Original Sales Distribution')
plt.xlabel('Sales')
plt.ylabel('Frequency')

# Log-transformed sales distribution
plt.subplot(1, 2, 2)
sns.histplot(df_active['log_sales'], bins=50, kde=True, color='lightgreen')
plt.title('Log-Transformed Sales Distribution')
plt.xlabel('Log(Sales + 1)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:

# Ensure 'log_segment' exists and is correctly ordered
segment_order = ['Very Low', 'Low-Mid', 'Mid-High', 'Very-High']

# Calculate average MAPE by log segment (using df_active)
mape_by_segment = df_active.groupby('log_segment')['mape'].mean().reindex(segment_order)

# Create the plot
plt.figure(figsize=(10, 6))
ax = sns.countplot(data=df_active, x='log_segment', order=segment_order, palette='viridis')

# Annotate each bar with count and MAPE
for p, segment_name in zip(ax.patches, segment_order):
    height = p.get_height()
    xpos = p.get_x() + p.get_width() / 2.

    # Add count label above the bar
    ax.text(xpos, height + 50, f'{int(height)}', ha='center', fontsize=11, fontweight='bold')

    # Add MAPE label inside the bar
    mape_val = mape_by_segment.get(segment_name, np.nan)
    ax.text(xpos, height / 2, f'MAPE: {mape_val:.1f}%', ha='center', va='center',
            fontsize=10, color='white', fontweight='bold')

# Formatting
plt.title("Active Product Count by Log-Based Segment with Average MAPE")
plt.xlabel("Log-Based Sales Segment")
plt.ylabel("Count of Records")
plt.tight_layout()
plt.show()



* High volume products have the lowest average MAPE —  forecasts are relatively more accurate.
* Low volume products have a higher MAPE — forecasts are less accurate.


**** train separate models or tune differently for each segment

In [ ]:

plt.figure(figsize=(12, 6))
sns.boxplot(data=df_active, x='log_segment', y='mape')
plt.title('MAPE Distribution by Log Segment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_active, x='pf_segmentation', y='mape', palette='Set3')
plt.title('MAPE Distribution by Product Family Segmentation')
plt.xlabel('PF Segmentation')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(data=df_active, x='market_segmentation', y='mape', palette='Pastel1')
plt.title('MAPE Distribution by Market Segmentation')
plt.xlabel('Market Segmentation')
plt.ylabel('MAPE (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Encode segmentation columns
encoded_df = df_active.copy()
segmentation_cols = ['market_segmentation', 'pf_segmentation', 'gmid_segmentation', 'log_segment']
encoded_df[segmentation_cols] = encoded_df[segmentation_cols].astype('category').apply(lambda x: x.cat.codes)

correlation_matrix = encoded_df[['mape'] + segmentation_cols].corr()
correlation_matrix['mape']

In [ ]:
heatmap_df = (
    df_all.groupby(['pf_segmentation', 'market_segmentation'])['mape']
    .mean()
    .unstack()
)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_df, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Average MAPE by PF & Market Segmentation')
plt.xlabel('Market Segmentation')
plt.ylabel('PF Segmentation')
plt.tight_layout()
plt.show()



PF Segmentation	Market Segmentation	Avg MAPE (%)	Insight
Secure	Regular MCO	129.29	🚨 Highest error — this intersection struggles most
Secure	Major MCO	79.67	Moderate error, but still notable
Secure	Top MCO	43.62	🔹 Best performance in the Secure group
Supply Growth	Major MCO	116.71	🚨 Very high — worth deeper review
Supply Growth	Regular MCO	83.62	Moderate volatility
Supply Growth	Top MCO	37.59	🟢 Strongest forecast accuracy overall

What You Can Do With This
1️⃣ Targeted Model Calibration
Segment your ML pipeline based on these intersections. For example:

Use different hyperparameters or ensemble strategies for Secure + Regular MCO

Isolate Supply Growth + Top MCO as a benchmark or stability zone

2️⃣ Feature Engineering
Include these combined segmentations (e.g. Secure_Regular_MCO) as categorical features in your predictive model. That way, model training accounts for historical error volatility per cluster.

3️⃣ Anomaly Buffering
For high-MAPE cells like Secure + Regular MCO, consider:

Looser prediction intervals

Conservative forecasting methods

Override rules during promotional cycles or volatile months

Strategic Implications
Regular MCO + Secure is a red flag. It may need separate handling in forecasting: capped errors, exception rules, even exclusion from some training data.

Supply Growth + Top MCO behaves well—this could serve as a model calibration baseline or be trusted with more aggressive forecasting.

You might consider:

Segmenting models by seg_combo

Flagging volatile clusters for anomaly detection

Designing smarter fallback logic for the worst-performing segments

In [ ]:
#Combine segmentation columns into hybrid features
df_active['seg_combo'] = df_all['pf_segmentation'] + '_' + df_all['market_segmentation']


In [ ]:
# Create lag features
df_active = df_active.sort_values(by=['gmid', 'year', 'month'])
df_active['sales_lag_3'] = df_active.groupby('gmid')['sales'].shift(3)
df_active['forecast_lag_3'] = df_active.groupby('gmid')['forecast'].shift(3)

df_active['sales_lag_3'] = df_active['sales_lag_3'].fillna(0)
df_active['forecast_lag_3'] = df_active['forecast_lag_3'].fillna(0)


In [ ]:
df_active.head()

# Machine Learning

### Train Test Split

In [ ]:
features = ['sales_lag_3', 'forecast_lag_3', 'log_sales', 'pf_segmentation', 'market_segmentation']
X = pd.get_dummies(df_active[features])
y = df_active['sales']  # or forecast


In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

### Scalling

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

In [ ]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_train_scaled.head()

In [ ]:
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)
X_test_scaled.head()

#### Linear regression

In [ ]:

# Initialize the model
lin_reg = LinearRegression()

# Train the model
lin_reg.fit(X_train_scaled, y_train)

# Predict
pred_lin = lin_reg.predict(X_test_scaled)

# Evaluate performance
r2_lin = r2_score(y_test, pred_lin)
mae_lin = mean_absolute_error(y_test, pred_lin)
rmse_lin = root_mean_squared_error(y_test, pred_lin)

print(f"R² Score: {r2_lin:.3f}")
print(f"MAE: {mae_lin:.2f}")
print(f"RMSE: {rmse_lin:.2f}")


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, pred_lin, alpha=0.7, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')

plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales (Linear Regression)')
plt.title(f'Linear Regression Forecast\nR²: {r2_lin:.2f} | MAE: {mae_lin:.2f}')
plt.grid(True)
plt.tight_layout()
plt.show()

#### KNN

In [ ]:
# Train KNN model
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train_scaled, y_train)

# Predict on test data
pred_knn = knn.predict(X_test_scaled)

# Evaluate
print("MAE", mean_absolute_error(pred_knn, y_test))
print("RMSE", root_mean_squared_error(pred_knn, y_test))
print("R2 score", knn.score(X_test_scaled, y_test))

In [ ]:
r2_knn = r2_score(y_test, pred_knn)
mae_knn = mean_absolute_error(y_test, pred_knn)


plt.figure(figsize=(8,6))
plt.scatter(y_test, pred_knn, alpha=0.7, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')

plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales (KNN)')
plt.title(f'Actual vs Predicted Values with KNN\nR²: {r2_knn:.2f} | MAE: {mae_knn:.2f}')
plt.grid(True)
plt.tight_layout()
plt.show()


#### RandomForest

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=20)
rf_model.fit(X_train_scaled, y_train)
pred_rf = rf_model.predict(X_test_scaled)

print("MAE", mean_absolute_error(pred_rf, y_test))
print("RMSE", root_mean_squared_error(pred_rf, y_test))
print("R2 score", rf_model.score(X_test_scaled, y_test))

In [ ]:
# Plot actual vs predicted

r2 = r2_score(y_test, pred_rf)
mae = mean_absolute_error(y_test, pred_rf)

plt.figure(figsize=(8,6))
plt.scatter(y_test, pred_rf, alpha=0.7, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # Perfect fit line

plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title(f'Actual vs Predicted (RandomForest)\nR²: {r2:.2f} | MAE: {mae:.2f}')
plt.grid(True)
plt.tight_layout()
plt.show()


#### Gradient Boosting

In [ ]:
gb_reg = GradientBoostingRegressor(max_depth=6, n_estimators=100)
gb_reg.fit(X_train_scaled, y_train)

# Predict and evaluate
pred_gb = gb_reg.predict(X_test_scaled)
mae_pred_gb = mean_absolute_error(y_test, pred_gb)
rmse_pred_gb = root_mean_squared_error(y_test, pred_gb)
r2_pred_gb = gb_reg.score(X_test_scaled, y_test)

print("MAE:", mae_pred_gb)
print("RMSE:", rmse_pred_gb)
print("R2 Score:", r2_pred_gb)

In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(8,6))
plt.scatter(y_test, pred_gb, alpha=0.7, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # Perfect fit line

plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title(f'Actual vs Predicted (Gradient Boosting)\nR²: {r2_pred_gb:.2f} | MAE: {mae_pred_gb:.2f}')
plt.grid(True)
plt.tight_layout()
plt.show()

#### XGBoost

In [ ]:
# Initialize XGBoost model
xgb_reg = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    objective='reg:squarederror',
    random_state=42
)

xgb_reg.fit(X_train, y_train)

# Evaluate
pred_xgb = xgb_reg.predict(X_test)


print("XGBoost MAE:", mean_absolute_error(y_test, pred_xgb))
print("XGBoost RMSE:", root_mean_squared_error(y_test, pred_xgb))
print("XGBoost R²:", r2_score(y_test, pred_xgb))


#### AdaBoost Regressor

In [ ]:
# Initialize AdaBoost
ada_reg = AdaBoostRegressor(n_estimators=100, learning_rate=0.5, random_state=42)

# Train
ada_reg.fit(X_train_scaled, y_train)

# Predict and Evaluate
pred_ada = ada_reg.predict(X_test_scaled)
print("AdaBoost MAE:", mean_absolute_error(y_test, pred_ada))
print("AdaBoost RMSE:", root_mean_squared_error(y_test, pred_ada))
print("AdaBoost R²:", r2_score(y_test, pred_ada))


#### Bagging Regressor

In [ ]:

# Initialize Bagging
bag_reg = BaggingRegressor(n_estimators=100, random_state=42)

# Train
bag_reg.fit(X_train_scaled, y_train)

# Predict and Evaluate
pred_bag = bag_reg.predict(X_test_scaled)
print("Bagging MAE:", mean_absolute_error(y_test, pred_bag))
print("Bagging RMSE:", mean_squared_error(y_test, pred_bag, squared=False))
print("Bagging R²:", r2_score(y_test, pred_bag))


In [ ]:

# Define models and predictions
model_names = ['Linear', 'KNN', 'Random Forest', 'Gradient Boosting','XGBoost', 'AdaBoost', 'Bagging']
models = [lin_reg, knn, rf_model, gb_reg, xgb_reg, ada_reg, bag_reg]

# Collect metrics dynamically
rmse_train = [
    root_mean_squared_error(y_train, model.predict(X_train if model == xgb_reg else X_train_scaled))
    for model in models
]
rmse_test = [
    root_mean_squared_error(y_test, model.predict(X_test if model == xgb_reg else X_test_scaled))
    for model in models
]
r2_train = [
    r2_score(y_train, model.predict(X_train if model == xgb_reg else X_train_scaled))
    for model in models
]

r2_test = [
    r2_score(y_test, model.predict(X_test if model == xgb_reg else X_test_scaled))
    for model in models
]



# Bar chart setup
x = np.arange(len(model_names))
width = 0.35

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# RMSE comparison
rects1 = ax1.bar(x - width/2, rmse_train, width, label='Train', color='mediumseagreen')
rects2 = ax1.bar(x + width/2, rmse_test,  width, label='Test',  color='cornflowerblue')
ax1.set_ylabel('RMSE')
ax1.set_title('Train vs Test RMSE by Model')
ax1.set_xticks(x)
ax1.set_xticklabels(model_names, rotation=45)
ax1.legend()

# R² comparison
rects3 = ax2.bar(x - width/2, r2_train, width, label='Train', color='salmon')
rects4 = ax2.bar(x + width/2, r2_test,  width, label='Test',  color='orange')
ax2.set_ylabel('R² Score')
ax2.set_title('Train vs Test R² by Model')
ax2.set_xticks(x)
ax2.set_xticklabels(model_names, rotation=45)
ax2.legend()

# Annotate bars with values
def annotate_bars(ax, rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}', xy=(rect.get_x() + rect.get_width()/2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom')

annotate_bars(ax1, rects1)
annotate_bars(ax1, rects2)
annotate_bars(ax2, rects3)
annotate_bars(ax2, rects4)

plt.tight_layout()
plt.show()


Gradient Boosting stands out as the most promising candidate for tuning:
metric	Train	Test	Insight
RMSE	4.97	749.44	Lowest test error — excellent generalization
R² Score	1.00	1.00	Perfect fit — but worth validating for overfittingM
the tiny train RMSE suggests it’s fitting the training data extremely well.

The low test RMSE and perfect R² imply it’s generalizing better than other models.

Compared to Random Forest and Bagging (which also show strong R²), Gradient Boosting has a much lower test RMSE, making it more efficient and precise.

That smaller difference between training and testing metrics in AdaBoost suggests it has better generalization and less overfitting compared to some other models.
AdaBoost isn’t overly memorizing the training data like some models that hit perfect R² on train but wobble on test.

🔹 While it may not have the absolute lowest RMSE, it could be more stable and reliable across unseen data.

🔹 It’s a strong candidate if your goal is robustness rather than squeezing every last drop of accuracy.

In [ ]:

# Initialize models
gb_model = GradientBoostingRegressor(random_state=42)
ada_model = AdaBoostRegressor(random_state=42)

# Run 5-fold cross-validation
gb_scores = cross_val_score(gb_model, X, y, cv=5, scoring='neg_root_mean_squared_error')
ada_scores = cross_val_score(ada_model, X, y, cv=5, scoring='neg_root_mean_squared_error')

# Convert to positive RMSE
print("Gradient Boosting CV RMSE:", -gb_scores.mean())
print("AdaBoost CV RMSE:", -ada_scores.mean())


Gradient Boosting is outperforming AdaBoost quite decisively in cross-validation, with a much lower RMSE. While AdaBoost looked promising in terms of stability between train and test, Gradient Boosting has the edge in predictive precision across folds.
Gradient Boosting is your most efficient learner for this task, especially if your goal is minimizing forecast error.

📉 AdaBoost is more cautious, but that extra bias might not be worth the hit in accuracy (~+1200 units of RMSE).

### Hyperparameter Tuning

#### Gradient Boosting

In [ ]:
#Define hyperparameter distribution (use reasonable ranges to avoid too large a search space)
Parameter_Trials = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.1, 0.5],
    'max_depth': [10, 14],
    'min_samples_split': [3, 6],
    'min_samples_leaf': [1, 2],
    'subsample': [0.3, 0.5]
}

# Perform RandomizedSearchCV
model = RandomizedSearchCV(gb_reg, Parameter_Trials, n_iter = 10, cv = 5, n_jobs = -1)

# Fit the model
model.fit(X_train_scaled,y_train)

In [ ]:
# Get the best hyperparameters
model.best_params_

In [ ]:
# Get the best model
best_model = model.best_estimator_

In [ ]:
# Evaluate the best model on the test set
best_pred_gb = best_model.predict(X_test_scaled)

print("MAE", mean_absolute_error(best_pred_gb, y_test))
print("RMSE", mean_squared_error(best_pred_gb, y_test, squared=False))
print("R2 score", best_model.score(X_test_scaled, y_test))

In [ ]:
# Define metrics
metrics = ["MAE", "RMSE", "R²"]
baseline_values = [
    mean_absolute_error(pred_gb, y_test),
    mean_squared_error(pred_gb, y_test, squared=False),
    gb_reg.score(X_test_scaled, y_test)
]
tuned_values = [
    mean_absolute_error(y_test, best_pred_gb),
    mean_squared_error(y_test, best_pred_gb, squared=False),
    best_model.score(X_test_scaled, y_test)
]

# Set position of bars
x = np.arange(len(metrics))
width = 0.3

# Create figure
plt.figure(figsize=(8, 5))
plt.bar(x - width/2, baseline_values, width, label="Baseline Gradient Boosting", color="grey")
plt.bar(x + width/2, tuned_values, width, label="Tuned GB", color="Green")

# Add labels
plt.xticks(x, metrics)
plt.ylabel("Score")
plt.title("Performance Comparison: Baseline vs Hyperparameter Tuning Gradient Boosting")
plt.legend(loc="upper right", ncol=1)  # Horizontal legend

# Show values on bars
for i in range(len(metrics)):
    plt.text(i - width/2, baseline_values[i] + 0.02, f"{baseline_values[i]:.2f}", ha='center', fontsize=12)
    plt.text(i + width/2, tuned_values[i] + 0.02, f"{tuned_values[i]:.2f}", ha='center', fontsize=12)

# Show chart
plt.show()


In [ ]:
print("Train R²:", best_model.score(X_train_scaled, y_train))


#### AdaBoost

In [ ]:
# Define custom base estimator
tree = DecisionTreeRegressor()

# Initialize AdaBoost with new 'estimator' parameter
ada_reg = AdaBoostRegressor(estimator=tree, random_state=42)

# Use double underscore to access DecisionTreeRegressor parameters
ada_param = {
    'n_estimators': [15, 20, 25, 30],
    'learning_rate': [0.01, 0.5, 1.0, 1.5],
    'estimator__max_leaf_nodes': [400, 450, 500, None],
    'estimator__max_depth': [5, 10, 15]
}

# Set up GridSearchCV
ada_search = GridSearchCV(
    estimator=ada_reg,
    param_grid=ada_param,
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Fit the model
ada_search.fit(X_train_scaled, y_train)

In [ ]:
# Get the best hyperparameters
ada_search.best_params_

In [ ]:
# Get the best model
best_model = ada_search.best_estimator_

In [ ]:
# Evaluate the best model on the test set
best_pred_ada = best_model.predict(X_test_scaled)

print("MAE", mean_absolute_error(best_pred_ada, y_test))
print("RMSE", mean_squared_error(best_pred_ada, y_test, squared=False))
print("R2 score", ada_search.score(X_test_scaled, y_test))

In [ ]:


# Baseline scores
mae_baseline = mean_absolute_error(y_test, pred_ada)
rmse_baseline = mean_squared_error(y_test, pred_ada, squared=False)
r2_baseline = r2_score(y_test, pred_ada)

# Tuned scores
mae_tuned = mean_absolute_error(y_test, best_pred_ada)
rmse_tuned = mean_squared_error(y_test, best_pred_ada, squared=False)
r2_tuned = r2_score(y_test, best_pred_ada)

# Bar setup
metrics = ['MAE', 'RMSE', 'R²']
baseline = [mae_baseline, rmse_baseline, r2_baseline]
tuned = [mae_tuned, rmse_tuned, r2_tuned]

x = np.arange(len(metrics))
width = 0.35

#Plot
plt.figure(figsize=(10, 6))
plt.bar(x - width/2, baseline, width, label='Baseline AdaBoost', color='slategray')
plt.bar(x + width/2, tuned, width, label='Tuned AdaBoost', color='mediumseagreen')

# Labels
plt.xticks(x, metrics, fontsize=12)
plt.ylabel("Metric Value", fontsize=12)
plt.title("📊 AdaBoost Performance: Before vs After Tuning", fontsize=14)
plt.legend()

# Annotate bars
for i in range(len(metrics)):
    plt.text(i - width/2, baseline[i] + 0.01, f"{baseline[i]:.2f}", ha='center', va='bottom', fontsize=11)
    plt.text(i + width/2, tuned[i] + 0.01, f"{tuned[i]:.2f}", ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()


### New results based on the model predictions

In [ ]:
df.to_csv('file_0507.csv', index=False)